In [1]:
!docker --version

Docker version 24.0.5, build 24.0.5-0ubuntu1~22.04.1


In [3]:
!docker ps | grep sonarqube

## Run sonarqube

In [ ]:
!docker run -d --name sonarqube -p 9000:9000 sonarqube

## Setting variables

In [94]:
%env SONARQUBE_URL=http://localhost:9000
%env USERNAME=admin
%env PASSWORD=sonar
%env YOUR_PROJECT_KEY=fastjson
%env YOUR_PROJECT_NAME=fastjson
%env YOUR_PROJECT_BRANCH="branch_1.2.24"

env: SONARQUBE_URL=http://localhost:9000
env: USERNAME=admin
env: PASSWORD=sonar
env: YOUR_PROJECT_KEY=fastjson
env: YOUR_PROJECT_NAME=fastjson
env: YOUR_PROJECT_BRANCH="branch_1.2.24"


## Creating user token

In [67]:
API_ENDPOINT="$SONARQUBE_URL/api/user_tokens/generate"
!curl -u $USERNAME:$PASSWORD -X POST $API_ENDPOINT -d name=automation_token > sonar.json
!jq -r '.token' sonar.json > token.txt
TOKEN = open('token.txt', 'r').read()
%env TOKEN=$TOKEN
!rm token.txt sonar.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178    0   157  100    21    631     84 --:--:-- --:--:-- --:--:--   717


In [121]:
!echo $TOKEN

squ_6427907900e64e54ec4daf52a372252ee9a873c1


## Create project

In [95]:
!curl -u "$USERNAME":"$PASSWORD" -X POST "$SONARQUBE_URL/api/projects/create?name=$YOUR_PROJECT_NAME&project=$YOUR_PROJECT_KEY&branch=$YOUR_PROJECT_BRANCH"

{"project":{"key":"fastjson","name":"fastjson","qualifier":"TRK","visibility":"public"}}

## Change directory

In [117]:
%env BESMAN_ARTIFACT_DIR="/home/arunsuresh/fastjson"

env: BESMAN_ARTIFACT_DIR="/home/arunsuresh/fastjson"


## Run Scan
Move to your project directory to perform the scan

### For maven builds

%env BESMAN_ARTIFACT_DIR="/home/arunsuresh/fastjson"
!echo "BESMAN_ARTIFACT_DIR=$BESMAN_ARTIFACT_DIR"
%cd "$BESMAN_ARTIFACT_DIR"
!ls
<!-- !mvn clean verify sonar:sonar -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.projectName="$YOUR_PROJECT_NAME" -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.login="$TOKEN" -->
!mvn clean install
!
%cd -

### For gradle builds

Running an analysis with Gradle is straighforward. You just need to declare the `org.sonarqube` plugin in your `build.gradle` or `build.gradle.kts` file:

**build.gradle**
```json
plugins {
  id "org.sonarqube" version "5.0.0.4638"
}
```

**build.gradle.kts**

```json
plugins {
  id("org.sonarqube") version "5.0.0.4638"
}
```


In [ ]:
%cd "$BESMAN_ARTIFACT_DIR"
!ls
!./gradlew sonar -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.projectName='$YOUR_PROJECT_NAME' -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.token="$TOKEN"

### For others(JS, TS, Go, Python, PHP)

In [ ]:
!sonar-scanner -Dsonar.projectKey="$YOUR_PROJECT_KEY" -Dsonar.sources=. -Dsonar.host.url="$SONARQUBE_URL" -Dsonar.token="$TOKEN"